Naive Bayes

In [1]:
%matplotlib inline
import pyro
import torch
import numpy as np
import matplotlib.pyplot as plt
import pyro.optim as optim
import pyro.distributions as dist
from torch.distributions import constraints
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
from matplotlib import animation, rc
from IPython.display import HTML
import torch.nn as nn
from functools import partial
import pandas as pd
from pyro.contrib.autoguide import AutoDiagonalNormal
from pyro.infer import EmpiricalMarginal, SVI, Trace_ELBO, TracePredictive
from scipy.stats import norm
from scipy.stats import beta as betaP
import math
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
from collections import Counter

In [2]:
# enable validation (e.g. validate parameters of distributions)
assert pyro.__version__.startswith('0.3.1')
pyro.enable_validation(True)

# clear the param store in case we're in a REPL
pyro.clear_param_store()

pyro.set_rng_seed(1)
warnings.filterwarnings('ignore')

In [3]:
iris_df = pd.read_csv('iris.csv', sep=',')
x_train, x_test, y_train, y_test =  train_test_split(iris_df, iris_df[iris_df.columns[4]], test_size=0.2)

labl1 = len(x_train[x_train.class_dataset == 1])
labl2 = len(x_train[x_train.class_dataset == 2])
labl3 = len(x_train[x_train.class_dataset == 3])

print("Probs of lable 1: " + str(labl1/len(y_train)) + "\n\t lable 2: " + str(labl2/len(y_train)) + "\n\t lable 3: " + str(labl3/len(y_train)))

Probs of lable 1: 0.325
	 lable 2: 0.30833333333333335
	 lable 3: 0.36666666666666664


In [6]:
class_num = 3
attribute_num = 4
count = iris_df.shape[0]
mi_result = np.empty((class_num, attribute_num))
sigma_result = np.empty((class_num, attribute_num))

pair_m_s4class = np.zeros(((class_num, attribute_num, 2)))

for i in range(class_num):
    for j in range(attribute_num):
        pair_m_s4class[i][j][0] = np.mean(iris_df.iloc[i*50:(i+1)*50, j])
        pair_m_s4class[i][j][1] = np.std(iris_df.iloc[i*50:(i+1)*50, j])

pair_m_s4class

array([[[5.006     , 0.34894699],
        [3.418     , 0.37719491],
        [1.464     , 0.17176728],
        [0.244     , 0.10613199]],

       [[5.936     , 0.51098337],
        [2.77      , 0.31064449],
        [4.26      , 0.46518813],
        [1.326     , 0.19576517]],

       [[6.588     , 0.62948868],
        [2.974     , 0.31925538],
        [5.552     , 0.54634787],
        [2.026     , 0.27188968]]])

In [7]:
pair_m_s4attribute = np.zeros((attribute_num, 2))

for i in range(attribute_num):
    pair_m_s4attribute[i][0] = np.mean(iris_df.iloc[i])
    pair_m_s4attribute[i][1] = np.std(iris_df.iloc[i])

pair_m_s4attribute

array([[2.24      , 1.79844377],
       [2.1       , 1.67092789],
       [2.08      , 1.63878003],
       [2.08      , 1.57657857]])

In [13]:
def model(data): 
    mi = pyro.sample('mi' , dist.Normal(2., 1.5))   
    sigma = pyro.sample('sigma' , dist.Uniform(0., 10.))    
    with pyro.plate('data', len(data)):
        data_obs = torch.from_numpy(data.values.astype(np.float)).type(torch.FloatTensor) 
        pyro.sample("obs", dist.Normal(mi,  sigma) , obs = data_obs)    

def guide(data):  
    mi_loc =pyro.param('mi_loc', torch.tensor(0.))
    mi_scale = pyro.param('mi_scale', torch.tensor(1.), constraint=constraints.positive)  
    
    sigma_loc = pyro.param('sigma_loc', torch.tensor(1.),  constraint=constraints.positive)
    sigma_scale = pyro.param('sigma_scale', torch.tensor(0.05),  constraint=constraints.positive) 
    
    mi = pyro.sample('mi', dist.Normal(mi_loc, mi_scale))
    sigma = pyro.sample('sigma', dist.Normal(sigma_loc, sigma_scale))

In [14]:
def train(data):
    pyro.clear_param_store()
    num_iterations= 3000
    optim = pyro.optim.Adam({"lr": 0.01})
    svi = pyro.infer.SVI(model, guide, optim, loss=pyro.infer.Trace_ELBO(), num_samples=data.shape[0])
    t=tqdm(range(num_iterations))
    losses = []
    for j in range(num_iterations):
        loss = svi.step(data)
        losses.append(loss)
    return (svi, losses)

In [15]:
for i in range(class_num):
    for j in range(attribute_num):
        dataz = x_train[x_train.class_dataset == i+1]
        dataz = dataz[dataz.columns[j]]
        svi, loss = train(dataz)
        posterior = svi.run(dataz)        
        mi_result[i, j] = pyro.param("mi_loc").item()
        sigma_result[i, j] = pyro.param("sigma_loc").item()
        print(pyro.param("mi_loc"))
        print(pyro.param("sigma_loc"))

tensor(4.7544, requires_grad=True)
tensor(0.9739, grad_fn=<AddBackward0>)


tensor(3.3207, requires_grad=True)
tensor(0.3820, grad_fn=<AddBackward0>)


tensor(1.4684, requires_grad=True)
tensor(0.1679, grad_fn=<AddBackward0>)


tensor(0.2345, requires_grad=True)
tensor(0.1149, grad_fn=<AddBackward0>)


tensor(5.9418, requires_grad=True)
tensor(0.5671, grad_fn=<AddBackward0>)


...

tensor(1.3172, requires_grad=True)
tensor(0.2085, grad_fn=<AddBackward0>)


tensor(6.5113, requires_grad=True)
tensor(0.6663, grad_fn=<AddBackward0>)


tensor(2.9462, requires_grad=True)
tensor(0.3095, grad_fn=<AddBackward0>)


tensor(5.5563, requires_grad=True)
tensor(0.6129, grad_fn=<AddBackward0>)


tensor(2.0283, requires_grad=True)
tensor(0.3045, grad_fn=<AddBackward0>)


In [16]:
dataz = torch.cat((torch.zeros(labl1), torch.ones(labl2), torch.empty(labl3).fill_(2.)))
dataz

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [17]:
def modelz(data):
    alpha = torch.tensor(6.0)
    beta = torch.tensor(10.0)
    lable_probs = pyro.sample('lable_probs', dist.Beta(alpha, beta).expand([3]).independent(1))
    normalized_lable_probs = lable_probs / torch.sum(lable_probs)

    with pyro.plate('data', len(data)):
        pyro.sample('obs', dist.Categorical(probs=normalized_lable_probs), obs=data)

In [18]:
def guidez(data):
    alphas = pyro.param('alphas', torch.tensor(6.).expand([3]), constraint=constraints.positive)
    betas = pyro.param('betas', torch.tensor(10.).expand([3]), constraint=constraints.positive) 

    pyro.sample('lable_probs', dist.Beta(alphas, betas).independent(1))

In [25]:
# def print_progress():
#     alphas = pyro.param("alphas")
#     betas = pyro.param("betas")
    
#     print(alphas)
#     print(betas)
        
# adam_params = {"lr": 0.0005}
# optimizer = pyro.optim.Adam(adam_params)
# svi = SVI(model, guide, optimizer, loss=pyro.infer.Trace_ELBO())

# n_steps = 2501
# for step in range(n_steps):
#     svi.step(dataz)
#     if step % 100 == 0:
#         print_progress()
def train(data):
    pyro.clear_param_store()
    num_iterations= 3000
    optimz = pyro.optim.Adam({"lr": 0.001})
    svi = pyro.infer.SVI(modelz, guidez, optimz, loss=pyro.infer.Trace_ELBO(), num_samples=data.shape[0])
    t=tqdm(range(num_iterations))
    losses = []
    for j in range(num_iterations):
        loss = svi.step(data)
        losses.append(loss)
    return (svi, losses)

lable_probabs = np.ones(3)
svi, loss = train(dataz)

alphas = pyro.param("alphas")
betas = pyro.param("betas")
means = alphas / (alphas + betas)
normalized_means = means / torch.sum(means)
for i in range(3):
    lable_probabs[i] = normalized_means[i].double()
print("1: " + str(lable_probabs[0]) + ", 2:" + str(lable_probabs[1]) + ", 3:" + str(lable_probabs[2]))

# adam_params = {"lr": 0.0005}
# optimizer = pyro.optim.Adam(adam_params)
# svi = SVI(model, guide, optimizer, loss=pyro.infer.Trace_ELBO())

# n_steps = 2501
# for step in range(n_steps):
#     svi.step(data)

1: 0.3293001055717468, 2:0.3170788288116455, 3:0.35362106561660767


In [27]:
def normpdf(x, mean, sd):
    var = float(sd)**2
    denom = (2*math.pi*var)**.5
    num = math.exp(-(float(x)-float(mean))**2/(2*var))
    return num/denom

def get_max(array):
    return list(array).index(np.max(array))+1

def get_result(test_row):
    result = lable_probabs.copy()
    for i in range(class_num):
        for j in range(attribute_num):
            result[i] *= normpdf(test_row[j], mi_result[i][j], sigma_result[i][j])
    return get_max(result)

def get_results4test(test):
    result = []
    for i in range(len(test)):
        result.append(get_result(test.iloc[i, :].values))
    return result

In [28]:
x = get_results4test(x_test)
print(np.array(x))
print(y_test.values)

[1 2 2 1 3 2 3 1 1 3 2 1 3 2 2 1 2 2 1 1 2 2 3 1 3 2 1 1 2 3]
[1 2 2 1 3 2 3 1 1 3 2 1 3 2 2 1 2 2 1 1 2 2 2 1 3 2 1 1 2 3]


In [29]:
from sklearn.metrics import accuracy_score

In [30]:
accuracy_score(y_test.values, x)

0.9666666666666667